In [53]:
import pandas as pd 
import numpy as np
import tensorflow as tf
from keras.preprocessing.image import * 
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import os 
from tensorflow.keras.preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import * 
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from PIL import Image, ImageFile
from keras.callbacks import ModelCheckpoint

In [54]:
# 모델 불러오기
from tensorflow.keras.models import load_model
model = load_model("/content/drive/MyDrive/ttproject/seq_naked_model.h5")

In [55]:
model.compile(loss="categorical_crossentropy", optimizer="rmsprop", metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 214, 214, 32)      896       
                                                                 
 batch_normalization_4 (Batc  (None, 214, 214, 32)     128       
 hNormalization)                                                 
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 107, 107, 32)     0         
 2D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 107, 107, 32)      0         
                                                                 
 conv2d_4 (Conv2D)           (None, 105, 105, 64)      18496     
                                                                 
 batch_normalization_5 (Batc  (None, 105, 105, 64)    

In [56]:
# 데이터 형상 관련 상수 정의 
IMAGE_WIDTH = 216
IMAGE_HEIGHT = 216
IMAGE_SIZE = (IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNEL = 3

# 학습 모델과 동일한 batch size 설정
batch_size = 8

## 노출영상001 테스트

In [70]:
# 테스트 데이터 준비 
from natsort import natsorted

path="/content/drive/MyDrive/ttproject/tiktok_video/노출영상/노출영상001"
filenames = natsorted(os.listdir(path))

test_df = pd.DataFrame({"filename":filenames})  

In [71]:
nbsamples=test_df.shape[0]
earlystop = EarlyStopping(patience=10)
learning_rate_reduction=ReduceLROnPlateau(
                        monitor= "val_accuracy", 
                        patience = 2, 
                        factor = 0.5, 
                        min_lr=0.0001,
                        verbose=1)
checkpoint = ModelCheckpoint("weights.h5", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

callbacks = [earlystop, learning_rate_reduction, checkpoint]

In [72]:
test_datagen=ImageDataGenerator(rescale=1./255)
# 테스트 이미지니까, 사진 그대로 씀
test_generator=test_datagen.flow_from_dataframe(
    test_df,
    path,
    x_col= "filename",
    y_col= None,
    target_size = IMAGE_SIZE,
    class_mode = None,
    batch_size = batch_size,
    shuffle = False)

Found 11 validated image filenames.


In [73]:
# 예측 
predict=model.predict(test_generator, 
                      steps=nbsamples/batch_size, 
                      callbacks=callbacks)
test_df['prediction']=np.argmax(predict, axis=1)

# 실제 정답 레이블
category_list = [1] * 11                                
test_df['category'] = category_list

test_df['prediction']=test_df['prediction'].replace({0:'비노출',1:"노출"})
test_df['category']=test_df['category'].replace({0:'비노출',1:"노출"})
test_df

1/1 [==============================] - 0s 93ms/step


,filename,prediction,category
0,frame_0.jpg,비노출,노출
1,frame_1.jpg,비노출,노출
2,frame_2.jpg,비노출,노출
3,frame_3.jpg,비노출,노출
4,frame_4.jpg,비노출,노출
5,frame_5.jpg,비노출,노출
6,frame_6.jpg,비노출,노출
7,frame_7.jpg,비노출,노출
8,frame_8.jpg,비노출,노출
9,frame_9.jpg,비노출,노출


In [74]:
# 'prediction' 열과 'category' 열이 같으면 정답
test_df['correct'] = (test_df['prediction'] == test_df['category'])
accuracy = test_df['correct'].sum() / len(test_df)

print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 0.00%


## 노출영상002

In [75]:
# 테스트 데이터 준비 
from natsort import natsorted

path="/content/drive/MyDrive/ttproject/tiktok_video/노출영상/노출영상002"
filenames = natsorted(os.listdir(path))

test_df = pd.DataFrame({"filename":filenames})  

nbsamples=test_df.shape[0]
earlystop = EarlyStopping(patience=10)
learning_rate_reduction=ReduceLROnPlateau(
                        monitor= "val_accuracy", 
                        patience = 2, 
                        factor = 0.5, 
                        min_lr=0.0001,
                        verbose=1)
checkpoint = ModelCheckpoint("weights.h5", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

callbacks = [earlystop, learning_rate_reduction, checkpoint]

test_datagen=ImageDataGenerator(rescale=1./255)
# 테스트 이미지니까, 사진 그대로 씀
test_generator=test_datagen.flow_from_dataframe(
    test_df,
    path,
    x_col= "filename",
    y_col= None,
    target_size = IMAGE_SIZE,
    class_mode = None,
    batch_size = batch_size,
    shuffle = False)

Found 28 validated image filenames.


In [76]:
# 예측 
predict=model.predict(test_generator, 
                      steps=nbsamples/batch_size, 
                      callbacks=callbacks)
test_df['prediction']=np.argmax(predict, axis=1)

# 실제 정답 레이블
category_list = [0]*6 + [1]*22                               
test_df['category'] = category_list

test_df['prediction']=test_df['prediction'].replace({0:'비노출',1:"노출"})
test_df['category']=test_df['category'].replace({0:'비노출',1:"노출"})
test_df

3/3 [==============================] - 0s 66ms/step


,filename,prediction,category
0,frame_0.jpg,비노출,비노출
1,frame_1.jpg,비노출,비노출
2,frame_2.jpg,비노출,비노출
3,frame_3.jpg,비노출,비노출
4,frame_4.jpg,비노출,비노출
5,frame_5.jpg,비노출,비노출
6,frame_6.jpg,비노출,노출
7,frame_7.jpg,비노출,노출
8,frame_8.jpg,비노출,노출
9,frame_9.jpg,비노출,노출


In [77]:
# 'prediction' 열과 'category' 열이 같으면 정답
test_df['correct'] = (test_df['prediction'] == test_df['category'])
accuracy = test_df['correct'].sum() / len(test_df)

print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 28.57%


### 노출영상003


In [78]:
# 테스트 데이터 준비 
from natsort import natsorted

path="/content/drive/MyDrive/ttproject/tiktok_video/노출영상/노출영상003"
filenames = natsorted(os.listdir(path))

test_df = pd.DataFrame({"filename":filenames})  

nbsamples=test_df.shape[0]
earlystop = EarlyStopping(patience=10)
learning_rate_reduction=ReduceLROnPlateau(
                        monitor= "val_accuracy", 
                        patience = 2, 
                        factor = 0.5, 
                        min_lr=0.0001,
                        verbose=1)
checkpoint = ModelCheckpoint("weights.h5", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

callbacks = [earlystop, learning_rate_reduction, checkpoint]

test_datagen=ImageDataGenerator(rescale=1./255)
# 테스트 이미지니까, 사진 그대로 씀
test_generator=test_datagen.flow_from_dataframe(
    test_df,
    path,
    x_col= "filename",
    y_col= None,
    target_size = IMAGE_SIZE,
    class_mode = None,
    batch_size = batch_size,
    shuffle = False)

Found 16 validated image filenames.


In [81]:
# 예측 
predict=model.predict(test_generator, 
                      steps=nbsamples/batch_size, 
                      callbacks=callbacks)
test_df['prediction']=np.argmax(predict, axis=1)

# 실제 정답 레이블
category_list = [0]*4 + [1]*12                               
test_df['category'] = category_list

test_df['prediction']=test_df['prediction'].replace({0:'비노출',1:"노출"})
test_df['category']=test_df['category'].replace({0:'비노출',1:"노출"})
test_df

2/2 [==============================] - 0s 37ms/step


,filename,prediction,category
0,frame_0.jpg,비노출,비노출
1,frame_1.jpg,비노출,비노출
2,frame_2.jpg,비노출,비노출
3,frame_3.jpg,비노출,비노출
4,frame_4.jpg,비노출,노출
5,frame_5.jpg,노출,노출
6,frame_6.jpg,비노출,노출
7,frame_7.jpg,비노출,노출
8,frame_8.jpg,비노출,노출
9,frame_9.jpg,비노출,노출


In [82]:
# 'prediction' 열과 'category' 열이 같으면 정답
test_df['correct'] = (test_df['prediction'] == test_df['category'])
accuracy = test_df['correct'].sum() / len(test_df)

print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 31.25%


### 노출영상004

In [83]:
# 테스트 데이터 준비 
from natsort import natsorted

path="/content/drive/MyDrive/ttproject/tiktok_video/노출영상/노출영상004"
filenames = natsorted(os.listdir(path))

test_df = pd.DataFrame({"filename":filenames})  

nbsamples=test_df.shape[0]
earlystop = EarlyStopping(patience=10)
learning_rate_reduction=ReduceLROnPlateau(
                        monitor= "val_accuracy", 
                        patience = 2, 
                        factor = 0.5, 
                        min_lr=0.0001,
                        verbose=1)
checkpoint = ModelCheckpoint("weights.h5", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

callbacks = [earlystop, learning_rate_reduction, checkpoint]

test_datagen=ImageDataGenerator(rescale=1./255)
# 테스트 이미지니까, 사진 그대로 씀
test_generator=test_datagen.flow_from_dataframe(
    test_df,
    path,
    x_col= "filename",
    y_col= None,
    target_size = IMAGE_SIZE,
    class_mode = None,
    batch_size = batch_size,
    shuffle = False)

Found 16 validated image filenames.


In [84]:
# 예측 
predict=model.predict(test_generator, 
                      steps=nbsamples/batch_size, 
                      callbacks=callbacks)
test_df['prediction']=np.argmax(predict, axis=1)

# 실제 정답 레이블
category_list = [0]*4 + [1]*12                               
test_df['category'] = category_list

test_df['prediction']=test_df['prediction'].replace({0:'비노출',1:"노출"})
test_df['category']=test_df['category'].replace({0:'비노출',1:"노출"})
test_df

2/2 [==============================] - 0s 177ms/step


,filename,prediction,category
0,frame_0.jpg,노출,비노출
1,frame_1.jpg,노출,비노출
2,frame_2.jpg,노출,비노출
3,frame_3.jpg,비노출,비노출
4,frame_4.jpg,비노출,노출
5,frame_5.jpg,비노출,노출
6,frame_6.jpg,비노출,노출
7,frame_7.jpg,비노출,노출
8,frame_8.jpg,비노출,노출
9,frame_9.jpg,비노출,노출


In [85]:
# 'prediction' 열과 'category' 열이 같으면 정답
test_df['correct'] = (test_df['prediction'] == test_df['category'])
accuracy = test_df['correct'].sum() / len(test_df)

print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 6.25%


### 노출영상005

In [86]:
# 테스트 데이터 준비 
from natsort import natsorted

path="/content/drive/MyDrive/ttproject/tiktok_video/노출영상/노출영상005"
filenames = natsorted(os.listdir(path))

test_df = pd.DataFrame({"filename":filenames})  

nbsamples=test_df.shape[0]
earlystop = EarlyStopping(patience=10)
learning_rate_reduction=ReduceLROnPlateau(
                        monitor= "val_accuracy", 
                        patience = 2, 
                        factor = 0.5, 
                        min_lr=0.0001,
                        verbose=1)
checkpoint = ModelCheckpoint("weights.h5", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

callbacks = [earlystop, learning_rate_reduction, checkpoint]

test_datagen=ImageDataGenerator(rescale=1./255)
# 테스트 이미지니까, 사진 그대로 씀
test_generator=test_datagen.flow_from_dataframe(
    test_df,
    path,
    x_col= "filename",
    y_col= None,
    target_size = IMAGE_SIZE,
    class_mode = None,
    batch_size = batch_size,
    shuffle = False)

Found 10 validated image filenames.


In [87]:
# 예측 
predict=model.predict(test_generator, 
                      steps=nbsamples/batch_size, 
                      callbacks=callbacks)
test_df['prediction']=np.argmax(predict, axis=1)

# 실제 정답 레이블
category_list = [0]*5 + [1]*5                              
test_df['category'] = category_list

test_df['prediction']=test_df['prediction'].replace({0:'비노출',1:"노출"})
test_df['category']=test_df['category'].replace({0:'비노출',1:"노출"})
test_df

1/1 [==============================] - 0s 137ms/step


,filename,prediction,category
0,frame_0.jpg,노출,비노출
1,frame_1.jpg,노출,비노출
2,frame_2.jpg,노출,비노출
3,frame_3.jpg,노출,비노출
4,frame_4.jpg,비노출,비노출
5,frame_5.jpg,비노출,노출
6,frame_6.jpg,비노출,노출
7,frame_7.jpg,비노출,노출
8,frame_8.jpg,비노출,노출
9,frame_9.jpg,비노출,노출


In [88]:
# 'prediction' 열과 'category' 열이 같으면 정답
test_df['correct'] = (test_df['prediction'] == test_df['category'])
accuracy = test_df['correct'].sum() / len(test_df)

print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 10.00%
